In [1]:
import sys

def obter_taxa_spot(t, curva_juros):
    """
    Obtém a taxa de juros spot (anualizada) para uma determinada maturidade (em anos).

    A curva_juros é uma lista de tuplas: [(maturidade_limite, taxa), ...],
    ordenada pela maturidade.
    """
    for maturidade_limite, taxa in curva_juros:
        if t <= maturidade_limite:
            return taxa
    # Caso t seja maior que o último limite, retorna a última taxa (curva flat)
    return curva_juros[-1][1]

def calcular_preco_titulo(maturidade_anos, principal, taxa_cupom_anual, freq_cupom, curva_juros):
    """
    Calcula o preço de um título descontando seus fluxos de caixa pela
    curva de juros zero-cupom (spot).

    Premissa: As taxas de juros spot são compostas na mesma frequência
    dos cupons do título (semestralmente).
    """
    preco = 0
    pagamento_cupom = (principal * taxa_cupom_anual) / freq_cupom
    num_periodos = int(maturidade_anos * freq_cupom)

    for i in range(1, num_periodos + 1):
        # Tempo até o fluxo de caixa, em anos
        t = i / freq_cupom  
        
        # Obtém a taxa spot anualizada para esta maturidade
        taxa_spot = obter_taxa_spot(t, curva_juros)
        
        # Fator de desconto.
        # PV = CF / (1 + r/k)^(t*k) onde k=freq_cupom
        fator_desconto = 1 / ((1 + taxa_spot / freq_cupom)**(t * freq_cupom))
        
        # Define o fluxo de caixa (cupom ou cupom + principal)
        fluxo_caixa = pagamento_cupom
        if i == num_periodos:
            fluxo_caixa += principal
            
        preco += fluxo_caixa * fator_desconto
        
    return preco

def analisar_sensibilidade():
    """
    Executa a análise de sensibilidade completa para os dois títulos
    e imprime os resultados.
    """
    
    # --- 1. Parâmetros dos Títulos ---
    PRINCIPAL = 100000.0
    TAXA_CUPOM = 0.10  # 10% p.a.
    FREQ_CUPOM = 2     # Semestral
    MATURIDADE_1 = 3   # 3 anos
    MATURIDADE_2 = 7   # 7 anos

    # --- 2. Definição da Curva de Juros Base ---
    # Estrutura: [(maturidade_limite_em_anos, taxa_anualizada), ...]
    curva_base = [
        (1.0, 0.05),   # 5.0% para maturidades <= 1.0 ano
        (2.0, 0.055),  # 5.5% para maturidades > 1.0 e <= 2.0 anos
        (999.0, 0.06)  # 6.0% para todas maturidades > 2.0 anos (999 é um proxy para "infinito")
    ]

    # --- 3. Definição dos Choques (Shifts) na Curva ---
    shift_10bps = 0.001  # 10 basis points
    shift_200bps = 0.02   # 200 basis points (2.0%)

    # --- 4. Criação das Curvas de Juros Deslocadas ---
    # Deslocamento paralelo
    curva_10bps = [(t, r + shift_10bps) for t, r in curva_base]
    curva_200bps = [(t, r + shift_200bps) for t, r in curva_base]

    # --- 5. Cálculo dos Preços Base ---
    # Estes são os preços "corretos" com base na curva de juros fornecida.
    preco_base_t1 = calcular_preco_titulo(MATURIDADE_1, PRINCIPAL, TAXA_CUPOM, FREQ_CUPOM, curva_base)
    preco_base_t2 = calcular_preco_titulo(MATURIDADE_2, PRINCIPAL, TAXA_CUPOM, FREQ_CUPOM, curva_base)

    # --- 6. Cálculo dos Preços Pós-Choque de +10 bps ---
    preco_10bps_t1 = calcular_preco_titulo(MATURIDADE_1, PRINCIPAL, TAXA_CUPOM, FREQ_CUPOM, curva_10bps)
    preco_10bps_t2 = calcular_preco_titulo(MATURIDADE_2, PRINCIPAL, TAXA_CUPOM, FREQ_CUPOM, curva_10bps)

    # --- 7. Cálculo dos Preços Pós-Choque de +200 bps ---
    preco_200bps_t1 = calcular_preco_titulo(MATURIDADE_1, PRINCIPAL, TAXA_CUPOM, FREQ_CUPOM, curva_200bps)
    preco_200bps_t2 = calcular_preco_titulo(MATURIDADE_2, PRINCIPAL, TAXA_CUPOM, FREQ_CUPOM, curva_200bps)

    # --- 8. Apresentação dos Resultados ---
    print("--- Análise de Sensibilidade de Títulos Corporativos ---")
    print("\n*** PREMISSAS IMPORTANTES DA ANÁLISE ***")
    print("1. O preço de mercado fornecido ($ 104.033,67) é INCONSISTENTE com a")
    print("   curva de juros zero-cupom fornecida. Ao calcular o preço do título de 3 anos")
    print("   usando a curva, o valor obtido é de $ 110.988,45.")
    print("2. Esta análise DESCONSIDERA o preço de mercado informado e usa os preços")
    print("   calculados a partir da curva zero-cupom como 'Preço Base'.")
    print("3. Assumimos que as taxas da curva zero-cupom (spot) são compostas")
    print("   semestralmente, para coincidir com a frequência dos cupons.")
    print("4. Os 'aumentos de juros' são interpretados como um DESLOCAMENTO PARALELO")
    print("   (parallel shift) de toda a curva de juros zero-cupom.")
    print("**************************************************")

    print("\n--- Preços Base (Calculados pela Curva Spot) ---")
    print(f"Título 1 (3 anos): $ {preco_base_t1:,.2f}")
    print(f"Título 2 (7 anos): $ {preco_base_t2:,.2f}")

    print("\n--- Cenário 1: Aumento de 10 bps (0.10%) ---")
    mudanca1_10bps = preco_10bps_t1 - preco_base_t1
    mudanca2_10bps = preco_10bps_t2 - preco_base_t2
    print(f"Novo Preço Título 1 (3 anos): $ {preco_10bps_t1:,.2f} | Mudança: $ {mudanca1_10bps:,.2f}")
    print(f"Novo Preço Título 2 (7 anos): $ {preco_10bps_t2:,.2f} | Mudança: $ {mudanca2_10bps:,.2f}")

    print("\n--- Cenário 2: Aumento de 200 bps (2.00%) ---")
    mudanca1_200bps = preco_200bps_t1 - preco_base_t1
    mudanca2_200bps = preco_200bps_t2 - preco_base_t2
    print(f"Novo Preço Título 1 (3 anos): $ {preco_200bps_t1:,.2f} | Mudança: $ {mudanca1_200bps:,.2f}")
    print(f"Novo Preço Título 2 (7 anos): $ {preco_200bps_t2:,.2f} | Mudança: $ {mudanca2_200bps:,.2f}")

    print("\n--- Conclusão da Análise ---")
    print("Como esperado, o título com maturidade mais longa (7 anos) possui uma")
    print("sensibilidade (duration) maior às taxas de juros.")
    print(f"Para uma alta de 200 bps, o título de 7 anos perde $ {abs(mudanca2_200bps):,.2f},")
    print(f"enquanto o título de 3 anos perde $ {abs(mudanca1_200bps):,.2f}.")

# Executa a função principal
if __name__ == "__main__":
    analisar_sensibilidade()


--- Análise de Sensibilidade de Títulos Corporativos ---

*** PREMISSAS IMPORTANTES DA ANÁLISE ***
1. O preço de mercado fornecido ($ 104.033,67) é INCONSISTENTE com a
   curva de juros zero-cupom fornecida. Ao calcular o preço do título de 3 anos
   usando a curva, o valor obtido é de $ 110.988,45.
2. Esta análise DESCONSIDERA o preço de mercado informado e usa os preços
   calculados a partir da curva zero-cupom como 'Preço Base'.
3. Assumimos que as taxas da curva zero-cupom (spot) são compostas
   semestralmente, para coincidir com a frequência dos cupons.
4. Os 'aumentos de juros' são interpretados como um DESLOCAMENTO PARALELO
   (parallel shift) de toda a curva de juros zero-cupom.
**************************************************

--- Preços Base (Calculados pela Curva Spot) ---
Título 1 (3 anos): $ 110,981.03
Título 2 (7 anos): $ 122,738.79

--- Cenário 1: Aumento de 10 bps (0.10%) ---
Novo Preço Título 1 (3 anos): $ 110,692.43 | Mudança: $ -288.60
Novo Preço Título 2 (7 anos